In [ ]:
import sys
sys.path.append('../src')

from database import DataAnalysisDB
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Connect to database
db = DataAnalysisDB('../data/analysis.db')
db.connect()

print("✓ Connected to database")
print(f"Tables: {db.list_tables()}")

## 1. View All Player Stats

In [ ]:
# Query all player stats
df = db.query("SELECT * FROM yb_stats")
print(f"Total players: {len(df)}\n")
df

## 2. Top Players by Defeated

In [ ]:
top_defeated = db.query("""
    SELECT player_name, defeated, assist, damage
    FROM yb_stats
    ORDER BY defeated DESC
    LIMIT 10
""")

print("Top 10 Players by Defeated:\n")
top_defeated

In [ ]:
# Visualize top players by defeated
plt.figure(figsize=(12, 6))
plt.barh(top_defeated['player_name'], top_defeated['defeated'], color='skyblue')
plt.xlabel('Defeated', fontsize=12)
plt.ylabel('Player', fontsize=12)
plt.title('Top 10 Players by Defeated', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 3. Top Players by Damage

In [ ]:
top_damage = db.query("""
    SELECT player_name, damage, defeated, assist
    FROM yb_stats
    ORDER BY damage DESC
    LIMIT 10
""")

print("Top 10 Players by Damage:\n")
top_damage

## 4. Average Statistics

In [ ]:
avg_stats = db.query("""
    SELECT 
        ROUND(AVG(defeated), 2) as avg_defeated,
        ROUND(AVG(assist), 2) as avg_assist,
        ROUND(AVG(damage), 2) as avg_damage,
        ROUND(AVG(tank), 2) as avg_tank,
        ROUND(AVG(heal), 2) as avg_heal,
        ROUND(AVG(siege_damage), 2) as avg_siege_damage
    FROM yb_stats
""")

print("Average Player Statistics:\n")
avg_stats

## 5. Players with High Damage per Defeated

In [ ]:
efficiency = db.query("""
    SELECT 
        player_name,
        defeated,
        damage,
        ROUND(CAST(damage AS FLOAT) / NULLIF(defeated, 0), 0) as damage_per_defeated
    FROM yb_stats
    WHERE defeated > 0
    ORDER BY damage_per_defeated DESC
    LIMIT 10
""")

print("Top 10 Players by Damage Efficiency (Damage per Defeated):\n")
efficiency

## 6. Support Players Analysis (High Heal/Tank)

In [ ]:
support_players = db.query("""
    SELECT 
        player_name,
        heal,
        tank,
        (heal + tank) as total_support,
        defeated
    FROM yb_stats
    WHERE (heal + tank) > 0
    ORDER BY total_support DESC
    LIMIT 10
""")

print("Top 10 Support Players (Heal + Tank):\n")
support_players

## 7. Correlation Analysis

In [ ]:
# Get all numeric data for correlation
all_data = db.query("SELECT defeated, assist, damage, tank, heal, siege_damage FROM yb_stats")

# Calculate correlation matrix
correlation = all_data.corr()

# Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=1, fmt='.2f')
plt.title('Correlation Matrix of Player Statistics', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 8. Custom Queries

Use the database object to run your own SQL queries:

In [ ]:
# Example: Find players with more than 100 assists
high_assists = db.query("""
    SELECT player_name, assist, defeated, damage
    FROM yb_stats
    WHERE assist > 100
    ORDER BY assist DESC
""")

print("Players with 100+ Assists:\n")
high_assists

In [ ]:
# Close database connection when done
db.close()
print("✓ Database connection closed")